## Upload Books

In [24]:
import requests

url = "http://127.0.0.1:8080"
book_name = "Art of Coaching Script.json"
book_path = r"M:\Projects\ro2ya\casy_api\courses\Art of Coaching\Art of Coaching Script.json"
upload_type = "application/vnd.openxmlformats-officedocument.wordprocessingml.document"

payload = {}
files=[
  ('file',(book_name, open(book_path,'rb'), upload_type))
]
headers = {}

response = requests.request("POST", url, headers=headers, data=payload, files=files)

print(response.text)

{"filename":"Giving Constructive Feedback Script.json","location":"books/Giving Constructive Feedback Script.json"}


## Test Stream Text

In [13]:
import requests
import json

stream_url = 'http://localhost:8080/text'
data = {
    "text": "ما الذي يتكلم عنه الكتاب أو الكورس وما آثاره الإيجابية؟"
}  

headers = {'Content-Type': 'application/json'}

with requests.post(stream_url, data=json.dumps(data),headers=headers, stream=True) as response:
    if response.status_code == 200:
        for chunk in response:
            print(chunk.decode('utf-8'), end="")
    else:
        print("Failed to connect to the streaming service")


الكورس بيتكلم عن فن التوجيه والتدريب وكيفية تأثيره على أداء الموظفين والمديرين والشركة بشكل عام. ومن خلال تبني ونشر ثقافة الـ coaching، بيكون له تأثير إيجابي على تحسين أداء الموظفين والمديرين وزيادة إنتاجية الشركة. بالإضافة إلى ذلك، يساهم في تحسين علاقات الفريق وتعزيز التواصل بين المدير والفريق وبين الأقسام المختلفة. وأيضًا يساهم في تطوير مهارات الموظفين وتقدمهم الوظيفي.

## Test Stream Audio

In [22]:
import requests
import json
from audio import stream

stream_url = 'http://localhost:8080/audio'
data = {
    "text": "what is your name"
}

headers = {'Content-Type': 'application/json'}

with requests.post(stream_url, data=json.dumps(data),headers=headers, stream=True) as response:
    if response.status_code == 200:
        stream(response)
    else:
        print("Failed to connect to the streaming service")

print("Streaming completed.")

ChunkedEncodingError: Response ended prematurely

# Stream Text and Audio

In [30]:
import asyncio
import websockets
import json
from audio import stream
import subprocess
import base64

async def send_data_via_websocket():
    uri = "ws://localhost:8080/socket_audio"
    data = {
    "message": "what is your name"
}
    mpv_command = ["C:\\Program Files\\mpv\\mpv.exe",
                   "--no-cache", "--no-terminal", "--", "fd://0"]
    mpv_process = subprocess.Popen(
        mpv_command,
        stdin=subprocess.PIPE,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )

    audio = b""
    try:
        async with websockets.connect(uri) as websocket:
            await websocket.send(json.dumps(data))
            async for chunk in websocket:
                if type(chunk) == str:
                    print(chunk, end="")
                else:
                    if chunk is not None:
                        mpv_process.stdin.write(chunk)
                        mpv_process.stdin.flush()
                        audio += chunk
            if mpv_process.stdin:
                mpv_process.stdin.close()
            mpv_process.wait()
    except Exception as e:
        print(f"An error occurred: {e}")

loop = asyncio.get_event_loop()
if loop.is_running():
    asyncio.ensure_future(send_data_via_websocket())
else:
    loop.run_until_complete(send_data_via_websocket())